In [3]:
!rm -r Project
!git clone https://github.com/krishrajpal/Accurate-localization-of-key-points-Scleral-Spur-based-on-VGG19-on-ACA-and-measurement-of-relevant.git Project


rm: cannot remove 'Project': No such file or directory
Cloning into 'Project'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (276/276), done.
remote: Compressing objects: 100% (247/247), done.
remote: Total 323 (delta 45), reused 252 (delta 25), pack-reused 47
Receiving objects: 100% (323/323), 4.84 MiB | 3.89 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten

# Load your data
x_train = np.load('/content/Project/npy/x_train.npy')  # Assuming you have numpy arrays saved as 'train_data.npy'
y_train = np.load('/content/Project/npy/y_train.npy')  # Assuming you have numpy arrays saved as 'train_labels.npy'
x_test = np.load('/content/Project/npy/x_test.npy')  # Assuming you have numpy arrays saved as 'test_data.npy'
y_test = np.load('/content/Project/npy/y_test.npy')  # Assuming you have numpy arrays saved as 'test_labels.npy'

# Preprocess your data (normalize, etc.)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Define your model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(248, 248, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of VGG19
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
predictions = Dense(2)(x)  #Predict 2 coordinates

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
3/3 [==============================] - 86s 31s/step - loss: 11745.4766 - val_loss: 688.1715
Epoch 2/10
3/3 [==============================] - 85s 30s/step - loss: 1762.3556 - val_loss: 2887.8879
Epoch 3/10
3/3 [==============================] - 84s 30s/step - loss: 2127.8384 - val_loss: 4752.0474
Epoch 4/10
3/3 [==============================] - 85s 30s/step - loss: 2443.6162 - val_loss: 1600.0013
Epoch 5/10
3/3 [==============================] - 96s 36s/step - loss: 664.7277 - val_loss: 78.0888
Epoch 6/10
3/3 [==============================] - 85s 30s/step - loss: 174.3988 - val_loss: 686.4517
Epoch 7/10
3/3 [==============================] - 84s 30s/step - loss: 624.8579 - val_loss: 1162.5129
Epoch 8/10
3/3 [==============================] - 85s 30s/step - loss: 655.3856 - val_loss: 776.5710
Epoch 9/10
2/3 [===================>..........] - ETA: 23s - loss: 355.9516

In [6]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Predict on the test data
y_pred = model.predict(x_test)
# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
# Calculate mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

1/1 [==============================] - 14s 14s/step
Mean Squared Error: 76.23718108777136
Mean Absolute Error: 6.820522644940544


In [8]:
model.save('vgg19_coordinates_model_1.keras')

In [1]:
#VGG19 Netwok Using Augmentation
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten

# Load Data
x_train = np.load('/content/Project/npy/x_train.npy')  # Assuming you have numpy arrays saved as 'train_data.npy'
y_train = np.load('/content/Project/npy/y_train.npy')  # Assuming you have numpy arrays saved as 'train_labels.npy'
x_test = np.load('/content/Project/npy/x_test.npy')  # Assuming you have numpy arrays saved as 'test_data.npy'
y_test = np.load('/content/Project/npy/y_test.npy')  # Assuming you have numpy arrays saved as 'test_labels.npy'

# Define your model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(248, 248, 3))

# Fine-tune last few convolutional layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Add custom layers on top of VGG19
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(2)(x)  # Predict 2 coordinates

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with a lower learning rate
optimizer = Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Define callbacks for learning rate scheduling
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)

# Train the model with data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)

datagen.fit(x_train)

model.fit(datagen.flow(x_train, y_train, batch_size=32),
          epochs=30, #Original Epoch: 30
          steps_per_epoch=len(x_train) / 32,
          validation_data=(x_test, y_test),
          callbacks=[reduce_lr])

Epoch 1/30


KeyboardInterrupt: 

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Predict on the test data
y_pred = model.predict(x_test)
# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
# Calculate mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

1/1 [==============================] - 22s 22s/step
Mean Squared Error: 14654.128110078605
Mean Absolute Error: 107.9681409036412


In [14]:
#VGG19 Netwok Using Keras Layers
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

# Load your data
x_train = np.load('/content/Project/npy/x_train.npy')
y_train = np.load('/content/Project/npy/y_train.npy')
x_test = np.load('/content/Project/npy/x_test.npy')
y_test = np.load('/content/Project/npy/y_test.npy')

# Preprocess your data (normalize, etc.)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Define your model
def create_custom_vgg19():
    input_shape = (248, 248, 3)
    input_tensor = Input(shape=input_shape)

    # Construct the network
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv1_1')(input_tensor)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv1_2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='pool1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2_1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2_2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='pool2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='pool3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='pool4')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='pool5')(x)
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc6')(x)
    x = Dense(4096, activation='relu', name='fc7')(x)
    output = Dense(2, name='predictions')(x)  # Output layer with 2 neurons for coordinates
    return Model(inputs=input_tensor, outputs=output)

# Create the model
model = create_custom_vgg19()

# Compile the model
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')
# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
3/3 [==============================] - 290s 96s/step - loss: 1311727288320.0000 - val_loss: 135582.2656
Epoch 2/10
3/3 [==============================] - 278s 94s/step - loss: 57144.8789 - val_loss: 10411.3076
Epoch 3/10
3/3 [==============================] - 279s 94s/step - loss: 17316.3203 - val_loss: 10398.5957
Epoch 4/10
3/3 [==============================] - 279s 93s/step - loss: 17275.0137 - val_loss: 10290.1357
Epoch 5/10
3/3 [==============================] - 276s 94s/step - loss: 16983.6152 - val_loss: 9576.7734
Epoch 6/10
3/3 [==============================] - 274s 93s/step - loss: 15005.7686 - val_loss: 5045.8877
Epoch 7/10
3/3 [==============================] - 281s 97s/step - loss: 5269.4678 - val_loss: 19767.7129
Epoch 8/10
3/3 [==============================] - 273s 93s/step - loss: 5645.3975 - val_loss: 564.0797
Epoch 9/10
3/3 [==============================] - 273s 93s/step - loss: 4472.2354 - val_loss: 2343.1511
Epoch 10/10
3/3 [============================

In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Predict on the test data
y_pred = model.predict(x_test)
# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
# Calculate mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

1/1 [==============================] - 13s 13s/step
Mean Squared Error: 1041.7803505028771
Mean Absolute Error: 29.72676681069767


In [16]:
y_pred

array([[ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294],
       [ 25.82294, 102.25294]], dtype=float32)

In [17]:
y_test

array([[ 62, 187],
       [ 68, 113],
       [ 46, 141],
       [ 69, 115],
       [ 56, 133],
       [ 41, 138],
       [ 51, 126],
       [ 43, 136],
       [ 50, 127],
       [ 65, 118],
       [ 50, 140],
       [ 59, 122],
       [ 56, 131],
       [ 56, 128],
       [ 56, 131],
       [ 52, 133],
       [ 63, 126]])

In [5]:
!git config --global user.email "krishrajpal3@gmail.com"
!git config --global user.name "krishrajpal"